## Librerias y Funciones

Primero vamos a llamar las librerías que vamos a usar

In [1]:
import pickle
import numpy as np
import pandas as pd
import os
import datetime
import seaborn as sns

Algunas funciones que hemos creado para describir nuestras bases de datos.

In [2]:
def mostrar_dimensiones(_df):
    size = _df.size
    shape = _df.shape
    df_ndim = _df.ndim
    return print("""Dimensiones de la tabla: \nTamaño = {}\nForma = {}\nForma[0] x Forma[1] = {}
                 \nDimensiones de la tabla = {}\n\nDescripcion de las columnas de la tabla:\n""".
          format(size, shape, shape[0]*shape[1],df_ndim)), _df.info()

def crear_calidad(_df):
    completitud = []
    for i in list(_df.columns):
        vacios = _df[(_df[i]=="")|(_df[i].isna())|(_df[i].isnull())]
        count_vacios = len(vacios)
        medida = round((1-(count_vacios/len(_df)))*100,2)
        completitud.append([i,medida])
    
    calidad_df = pd.DataFrame(completitud, columns = ['Columna','Completitud de Col(%)'])
    unicidad=[]
    distintivo=[]
    type_col = []
    Moda= []
    for j in list(_df.columns):
        unico = len(_df[_df[j].duplicated()==False])
        temp_df = _df[_df[j].duplicated()==True]
        distinto = len(temp_df[j].unique())
        unicidad.append(unico-distinto)
        distintivo.append(distinto)
        type_col.append(_df[j].dtypes)
        Moda.append(_df[j].mode()[0])
    
    calidad_df['# Unicos']=unicidad
    calidad_df['# Distintos']=distintivo
    calidad_df['Tipo de Columna']=type_col
    calidad_df['Moda']=Moda
    
    return calidad_df

def calidad_num(_df):
    var_numericas = _df.describe().transpose()
    var_numericas.reset_index(inplace=True)
    var_numericas = var_numericas.rename(columns={'index':'Columna'})
    cal_df = crear_calidad(_df)

    descriptivos_num = cal_df.merge(var_numericas, on='Columna', how='left')
    descriptivos_num = descriptivos_num.replace(np.nan, '',regex=True)
 #   descriptivos_num = descriptivos_num[(descriptivos_num['Tipo de Columna']=='int32')|(descriptivos_num['Tipo de Columna']=='float64')]
    
    return descriptivos_num

def pareto_entry(_col,_df):
    cuenta = []
    unicos = []
    porcentaje = []
    for i in list(_df[_col].unique()):
        tempy = pd.DataFrame(_df[_col])
        cuenta.append(tempy[tempy[_col]==i].count()[0])
        unicos.append(i)
        porcentaje.append(round(tempy[tempy[_col]==i].count()[0]/len(_df),3)*100)
    
    pareto_df = pd.DataFrame(unicos, columns=[_col])
    pareto_df['Count'] = cuenta
    pareto_df['Percentage'] = porcentaje
    pareto_df = pareto_df.sort_values(by=['Count'], ascending=False)
    pareto_df['Cum_Percentage'] = round(100*(pareto_df['Count'].cumsum()/pareto_df['Count'].sum()),1)
    
    return pareto_df

## Lectura de los datos

Procedemos a la lectura de nuestros datos.

In [3]:
forecastLocation = "Datos Frubana\df_forecast_uniandes.pkl"
comprasLocation = "Datos Frubana\BAQ_compras.csv"
productosLocation = "Datos Frubana\Products_BAQ.csv"
wastePercentageLocation = "Datos Frubana\waste_percentage_by_age.csv"
ventasLocationFolder = os.listdir("Datos Frubana\\ventas")
ventasLocation = "Datos Frubana\\ventas"

In [4]:
compras = pd.read_csv(comprasLocation)
productos = pd.read_csv(productosLocation)
wastePercentageAge = pd.read_csv(wastePercentageLocation)

In [5]:
with open (forecastLocation, 'rb') as file:
    forecast = pickle.load(file)

ventas = pd.DataFrame()    
for i in ventasLocationFolder:
    with open(ventasLocation + "\\" + i, 'rb') as file:
        venticas = pickle.load(file)
    ventas = pd.concat([ventas, venticas])

### 1. Ventas

Comenzaremos por diagnosticar nuestra base de datos de ventas.

In [6]:
ventas.head(2)

nro_orden      fecha                            producto cantidad  \
184  18120194 2023-01-03   Papa Blanca Sucia Tamaño Mixto Kg  10.0000   
204  18131510 2023-01-03  Tomate Chonto Extramaduro Mixto Kg   1.0000   

        precio   descuento                           customer_id  \
184  2750.0000  46750.0000  1f1b98af-ee04-4849-a6cf-4fecdf7dba50   
204  4998.0000   1499.4000  8c868951-e6d6-4d63-9592-d4b08ec43f3c   

                                              sku product_id  \
184                 BAQ-FRU1-CAT6-234:304:750:770        770   
204  BAQ-FRU1-CAT104105-73423:168376:168377:92108      92108   

    product_quantity_x_step_unit product_step_unit product_unit sku_parent  \
184                     100.0000             10.00     Kilogram   No value   
204                       2.0000              2.00     Kilogram   No value   

     month  
184      1  
204      1

In [7]:
mostrar_dimensiones(ventas)

Dimensiones de la tabla: 
Tamaño = 7030156
Forma = (502154, 14)
Forma[0] x Forma[1] = 7030156
                 
Dimensiones de la tabla = 2

Descripcion de las columnas de la tabla:

<class 'pandas.core.frame.DataFrame'>
Index: 502154 entries, 184 to 35393
Data columns (total 14 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   nro_orden                     502154 non-null  object        
 1   fecha                         502154 non-null  datetime64[ns]
 2   producto                      502154 non-null  object        
 3   cantidad                      502154 non-null  object        
 4   precio                        502154 non-null  object        
 5   descuento                     502154 non-null  object        
 6   customer_id                   502154 non-null  object        
 7   sku                           502154 non-null  object        
 8   product_id                    50215

(None, None)

Nos aseguramos de obtener los tipos de datos correctos para cada columna.

In [8]:
ventasCopy = ventas.copy()

### 2. Compras

Ahora diagnosticaremos la base de datos de compras.

In [9]:
compras.head(2)

warehouse_code region_code       id delivery_date  product_id  \
0            BAQ         BAQ  8475266    2022-09-08         660   
1            BAQ         BAQ  8554498    2022-09-12         660   

                             sku                      name  supplier_id  \
0  BAQ-FRU1-CAT1-111:277:659:660  Granadilla Estándar - Kg           61   
1  BAQ-FRU1-CAT1-111:277:659:660  Granadilla Estándar - Kg           61   

    price  quantity  
0  5950.0       5.0  
1  5950.0       8.0

In [10]:
mostrar_dimensiones(compras)

Dimensiones de la tabla: 
Tamaño = 51030
Forma = (5103, 10)
Forma[0] x Forma[1] = 51030
                 
Dimensiones de la tabla = 2

Descripcion de las columnas de la tabla:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5103 entries, 0 to 5102
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   warehouse_code  5103 non-null   object 
 1   region_code     5103 non-null   object 
 2   id              5103 non-null   int64  
 3   delivery_date   5103 non-null   object 
 4   product_id      5103 non-null   int64  
 5   sku             5103 non-null   object 
 6   name            5103 non-null   object 
 7   supplier_id     5103 non-null   int64  
 8   price           5103 non-null   float64
 9   quantity        5103 non-null   float64
dtypes: float64(2), int64(3), object(5)
memory usage: 398.8+ KB


(None, None)

In [11]:
crear_calidad(compras)

Columna  Completitud de Col(%)  # Unicos  # Distintos  \
0  warehouse_code                  100.0         0            1   
1     region_code                  100.0         0            1   
2              id                  100.0      5103            0   
3   delivery_date                  100.0        30          308   
4      product_id                  100.0        15          137   
5             sku                  100.0        15          137   
6            name                  100.0        15          137   
7     supplier_id                  100.0        28          113   
8           price                  100.0       351          278   
9        quantity                  100.0       339          288   

  Tipo de Columna                                 Moda  
0          object                                  BAQ  
1          object                                  BAQ  
2           int64                              7741513  
3          object                           2023-01-30  
4           int64                                  770  
5          object        BAQ-FRU1-CAT6-234:304:750:770  
6          object  Papa Blanca Sucia Tamaño Mixto - KG  
7           int64                                  383  
8         float64                               2000.0  
9         float64                                 10.0

### 3. Waste Percentage Age

In [12]:
wastePercentageAge.head(2)

shelf life  age  waste_percentage (%)
0           1    1                 50.28
1           1    2                 75.27

In [13]:
mostrar_dimensiones(wastePercentageAge)

Dimensiones de la tabla: 
Tamaño = 23763
Forma = (7921, 3)
Forma[0] x Forma[1] = 23763
                 
Dimensiones de la tabla = 2

Descripcion de las columnas de la tabla:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7921 entries, 0 to 7920
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   shelf life            7921 non-null   int64  
 1   age                   7921 non-null   int64  
 2   waste_percentage (%)  7921 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 185.8 KB


(None, None)

In [14]:
crear_calidad(wastePercentageAge)

Columna  Completitud de Col(%)  # Unicos  # Distintos  \
0            shelf life                  100.0         0           89   
1                   age                  100.0         0           89   
2  waste_percentage (%)                  100.0      1296         1018   

  Tipo de Columna   Moda  
0           int64    1.0  
1           int64    1.0  
2         float64  100.0

### 4. Productos

In [15]:
productos.head(2)

product_id                            sku                      name  \
0         660  BAQ-FRU1-CAT1-111:277:659:660  Granadilla Estándar - Kg   
1         676  BAQ-FRU1-CAT1-123:280:665:676        Kiwi Estándar - Kg   

  category region_code  product_category_id  mean_shelf_life  \
0   Frutas         BAQ                    1                2   
1   Frutas         BAQ                    1                2   

   promised_lead_time  purchasing_unit buy_unit  weight_parameter_apricot  
0                 NaN              1.0       KG                       1.0  
1                 NaN              1.0       KG                       1.0

In [16]:
mostrar_dimensiones(productos)

Dimensiones de la tabla: 
Tamaño = 1507
Forma = (137, 11)
Forma[0] x Forma[1] = 1507
                 
Dimensiones de la tabla = 2

Descripcion de las columnas de la tabla:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137 entries, 0 to 136
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   product_id                137 non-null    int64  
 1   sku                       137 non-null    object 
 2   name                      137 non-null    object 
 3   category                  137 non-null    object 
 4   region_code               137 non-null    object 
 5   product_category_id       137 non-null    int64  
 6   mean_shelf_life           137 non-null    int64  
 7   promised_lead_time        32 non-null     float64
 8   purchasing_unit           137 non-null    float64
 9   buy_unit                  137 non-null    object 
 10  weight_parameter_apricot  137 non-null    float64
dtypes

(None, None)

In [17]:
crear_calidad(productos)

Columna  Completitud de Col(%)  # Unicos  # Distintos  \
0                 product_id                 100.00       137            0   
1                        sku                 100.00       137            0   
2                       name                 100.00       137            0   
3                   category                 100.00         0            3   
4                region_code                 100.00         0            1   
5        product_category_id                 100.00         0            1   
6            mean_shelf_life                 100.00         2            6   
7         promised_lead_time                  23.36         0            4   
8            purchasing_unit                 100.00         6            6   
9                   buy_unit                 100.00         0            2   
10  weight_parameter_apricot                 100.00         7           10   

   Tipo de Columna                           Moda  
0            int64                            100  
1           object  BAQ-FRU1-CAT1-111:277:659:660  
2           object        Acelga Estándar - Atado  
3           object                       Verduras  
4           object                            BAQ  
5            int64                              1  
6            int64                              2  
7          float64                            1.0  
8          float64                            1.0  
9           object                             KG  
10         float64                            1.0

### 5. Forecast

In [18]:
forecast.head(2)

created_at  timeframe  close_date warehouse  product_id  forecast
995604  2022-06-30          2  2022-07-02       BAQ         100      28.0
450926  2022-06-30          2  2022-07-02       BAQ         101      12.0

In [19]:
mostrar_dimensiones(forecast)

Dimensiones de la tabla: 
Tamaño = 7198122
Forma = (1199687, 6)
Forma[0] x Forma[1] = 7198122
                 
Dimensiones de la tabla = 2

Descripcion de las columnas de la tabla:

<class 'pandas.core.frame.DataFrame'>
Index: 1199687 entries, 995604 to 75933
Data columns (total 6 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   created_at  1199687 non-null  object 
 1   timeframe   1199687 non-null  int64  
 2   close_date  1199687 non-null  object 
 3   warehouse   1199687 non-null  object 
 4   product_id  1199687 non-null  int64  
 5   forecast    1199687 non-null  float64
dtypes: float64(1), int64(2), object(3)
memory usage: 64.1+ MB


(None, None)

In [20]:
crear_calidad(forecast)

Columna  Completitud de Col(%)  # Unicos  # Distintos Tipo de Columna  \
0  created_at                  100.0         0          308          object   
1   timeframe                  100.0         1           36           int64   
2  close_date                  100.0         0          313          object   
3   warehouse                  100.0         0            1          object   
4  product_id                  100.0         0          149           int64   
5    forecast                  100.0      2612         2448         float64   

         Moda  
0  2022-06-30  
1           2  
2  2022-08-18  
3         BAQ  
4      630219  
5         1.0

## Transformaciones de datos

Vamos a asegurarnos de que los tipos de datos de todas las bases de datos esten bien clasificadas.

In [21]:
ventasConvertDict = {'nro_orden': str, 'producto': str, 'cantidad': int, 'precio':float, 'descuento':float, 'customer_id' : str, 'sku' : str, 
                     'product_id': str, 'product_quantity_x_step_unit':int, 'product_step_unit':int, 'sku_parent':str}

comprasConvertDict = {'warehouse_code': str, 'region_code': str, 'id' : str, 'product_id': str, 'sku': str, 'name' : str, 'supplier_id': str,
                     'quantity' : int}

productosConvertDict = {'product_id':str, 'sku':str, 'name':str, 'category':str, 'region_code':str, 'product_category_id': str, 'buy_unit': str}

ventas = ventas.astype(ventasConvertDict)
compras = compras.astype(comprasConvertDict)
compras.delivery_date = pd.to_datetime(compras.delivery_date)
productos = productos.astype(productosConvertDict)

1. Se debe encontrar la manera de costear los productos que se vendieron
2. Se debe consolidar la base de datos de ventas con las tablas disponibles


In [22]:
ventas.fecha.min()

Timestamp('2022-07-01 00:00:00')

In [23]:
ventas.fecha.max()

Timestamp('2023-07-01 00:00:00')

Vemos que la base de datos de ventas contiene datos desde 7/1/2022 hasta 7/1/2023.

In [24]:
compras.delivery_date.min()

Timestamp('2022-07-26 00:00:00')

In [25]:
compras.delivery_date.max()

Timestamp('2023-08-29 00:00:00')

Por otro lado vemos que la base de datos de compras no tiene el mismo fragmento de tiempo que nuestra base de ventas. Acá haremos una lista de supocisiones:
1. Un producto dura en promedio 4 días inventariado desde su compra hasta su venta.
2. Usaremos fechas cruzadas para nuestra información de ventas. Por lo que el fragmento de tiempo que usaremos para el ejercicio será desde 7/26/2022 hasta el 7/1/2023. Sin embargo, dada nuestra suposición de los 4 días entre compra y venta, la base de datos de ventas comenzará desdel el 7/30/2022.
3. Aquellos productos de los que no tenemos información de compra (osea costo), no podrán ser costeados, y por lo tanto se retirarán de la base de datos consolidada.
4. Los costos operacionales relacionados con el inventario, mano de obra, etc, se estiman como el 14% de compras totales del día.

#### Acotemos las fechas de las bases de datos

In [26]:
ventas = ventas[ventas.fecha>=datetime.datetime(2022,7,30)]
compras = compras[compras.delivery_date<=datetime.datetime(2023,7,1)]

Ahora retiraremos de la base de datos aquellos productos que no podemos costear

In [27]:
productCostList = list(compras.sku.unique())

ventas.producto[~ventas.sku.isin(productCostList)].unique()

array(['Ajo Estandar Kg', 'Papa Blanca Sucia Tamaño Mixto Sucia x Bulto',
       'Guayaba Maduración Mixta Al por mayor',
       'Repollo Blanco Pequeño Unidad',
       'Cebollín Sucio Estándar Al por mayor',
       'Maracuyá Mixto Media Bolsa',
       'Cebolla Roja Mixta Mixta Desde 10kg',
       'Limón Tahití Estándar Al por mayor',
       'Papa Blanca Sucia Tamaño Mixto Bulto (48kg)',
       'Piña Golden Estandar Unidad (Tamaño 🏠)',
       'Lechuga Batavia Estandar Unidad x Canastilla',
       'Tomate Chonto Maduración Mixta Semi (Mediano) Al por mayor',
       'Cilantro Estandar Libra (Tamaño 🏠)',
       'Tomate Chonto Maduración Mixta Semi (Mediano) Kg (Tamaño 🏠)',
       'Cebolla Cabezona Blanca Sin Pelar Mixta Al por mayor',
       'Cebolla Roja Mixta Mixta Al por mayor',
       'Banano Criollo Estandar Kg (Tamaño 🏠)',
       'Ajo Estandar Libra (Tamaño 🏠)',
       'Cebolla Roja Mixta Mixta Kg (Tamaño 🏠)',
       'Guineo Verde KG Estandar Kg (Tamaño 🏠)',
       'Ahuyama Estándar

In [28]:
ventas = ventas[ventas.sku.isin(productCostList)]

In [29]:
len(ventas)

346891

Ahora agregamos nuestro precio ajustado de acuerdo a los costos operacionales.

In [30]:
compras['total_buy'] = compras.price * compras.quantity
compras['fix_price'] = compras.price * 1.14
compras['fix_total_buy'] = compras.fix_price * compras.quantity

In [31]:
compras.head()

warehouse_code region_code       id delivery_date product_id  \
0            BAQ         BAQ  8475266    2022-09-08        660   
1            BAQ         BAQ  8554498    2022-09-12        660   
2            BAQ         BAQ  8682054    2022-09-19        660   
3            BAQ         BAQ  8715673    2022-09-22        660   
4            BAQ         BAQ  8842178    2022-09-29        660   

                             sku                      name supplier_id  \
0  BAQ-FRU1-CAT1-111:277:659:660  Granadilla Estándar - Kg          61   
1  BAQ-FRU1-CAT1-111:277:659:660  Granadilla Estándar - Kg          61   
2  BAQ-FRU1-CAT1-111:277:659:660  Granadilla Estándar - Kg          61   
3  BAQ-FRU1-CAT1-111:277:659:660  Granadilla Estándar - Kg          19   
4  BAQ-FRU1-CAT1-111:277:659:660  Granadilla Estándar - Kg          61   

    price  quantity  total_buy  fix_price  fix_total_buy  
0  5950.0         5    29750.0     6783.0        33915.0  
1  5950.0         8    47600.0     6783.0        54264.0  
2  7500.0         2    15000.0     8550.0        17100.0  
3  9000.0         6    54000.0    10260.0        61560.0  
4  7200.0         3    21600.0     8208.0        24624.0

Ahora empezaremos con el proceso para costear.

In [32]:
start_date = datetime.datetime(2022,7,26)
end_date = datetime.datetime(2023,7,31)

date_range = pd.date_range(start=start_date, end = end_date, freq = "D")
date_table = pd.DataFrame({"Date": date_range})

In [33]:
compras_pivot = pd.pivot_table(compras, values=['price','fix_price'], index=['delivery_date', 'sku'], aggfunc="mean").reset_index()

Nos aseguramos que todos los días tengamos un precio registrado para cada producto.

In [34]:
sku_list = list(compras_pivot.sku)
costs = pd.DataFrame()
for i in sku_list:
    temp_costs = pd.merge(date_table, compras_pivot[compras_pivot.sku == i], how="left", left_on = "Date", right_on = "delivery_date")
    costs = pd.concat([costs, temp_costs], ignore_index=True)

In [35]:
costs = costs.ffill()

In [36]:
costs = costs[costs.delivery_date.notnull()]

In [37]:
costs.delivery_date.min()

Timestamp('2022-07-26 00:00:00')

In [38]:
costs.delivery_date.max()

Timestamp('2023-07-01 00:00:00')

In [39]:
costs.head()

Date delivery_date                          sku  fix_price   price
0 2022-07-26    2022-07-26  BAQ-FRU1-CAT1-47:67:151:152     3249.0  2850.0
1 2022-07-27    2022-07-26  BAQ-FRU1-CAT1-47:67:151:152     3249.0  2850.0
2 2022-07-28    2022-07-26  BAQ-FRU1-CAT1-47:67:151:152     3249.0  2850.0
3 2022-07-29    2022-07-29  BAQ-FRU1-CAT1-47:67:151:152     3249.0  2850.0
4 2022-07-30    2022-07-29  BAQ-FRU1-CAT1-47:67:151:152     3249.0  2850.0

In [40]:
costs_2 = costs.copy()

In [41]:
costs = costs[['Date','sku','price']]
costs.drop_duplicates(inplace=True)

In [42]:
len(costs_2)

1468789

In [43]:
len(costs)

57439

In [44]:
costs = costs[costs[['Date','sku']].duplicated()==False]

In [45]:
len(costs)

48768

In [46]:
crear_calidad(costs)

Columna  Completitud de Col(%)  # Unicos  # Distintos Tipo de Columna  \
0    Date                  100.0         0          371  datetime64[ns]   
1     sku                  100.0         0          146          object   
2   price                  100.0        67          529         float64   

                          Moda  
0          2023-06-08 00:00:00  
1  BAQ-FRU1-CAT1-14:49:115:116  
2                       2000.0

Ahora que hemos creado la tabla de costos. Procederemos a hacer transformaciones adicionales en ventas para poder crear nuestra base de datos consolidada.

In [47]:
ventas["totalVentasSinDescuento"] = ventas.cantidad*ventas.precio
ventas["totalVentasConDescuento"] = (ventas.cantidad*ventas.precio)-ventas.descuento

In [48]:
ventas["delivery_date"] = ventas.fecha - pd.to_timedelta(4, unit="d")

In [49]:
ventas.head(2)

nro_orden      fecha                            producto  cantidad  \
184  18120194 2023-01-03   Papa Blanca Sucia Tamaño Mixto Kg        10   
204  18131510 2023-01-03  Tomate Chonto Extramaduro Mixto Kg         1   

     precio  descuento                           customer_id  \
184  2750.0    46750.0  1f1b98af-ee04-4849-a6cf-4fecdf7dba50   
204  4998.0     1499.4  8c868951-e6d6-4d63-9592-d4b08ec43f3c   

                                              sku product_id  \
184                 BAQ-FRU1-CAT6-234:304:750:770        770   
204  BAQ-FRU1-CAT104105-73423:168376:168377:92108      92108   

     product_quantity_x_step_unit  product_step_unit product_unit sku_parent  \
184                           100                 10     Kilogram   No value   
204                             2                  2     Kilogram   No value   

     month  totalVentasSinDescuento  totalVentasConDescuento delivery_date  
184      1                  27500.0                 -19250.0    2022-12-30  
204      1                   4998.0                   3498.6    2022-12-30

In [50]:
ventasCopy = ventas.copy()

In [51]:
ventas = pd.merge(ventas, costs, how='left', left_on=['sku','delivery_date'], right_on=['sku','Date'])

In [52]:
len(ventasCopy)

346891

In [53]:
len(ventas)

346891

In [54]:
ventas.rename(columns={'price':'unit cost'}, inplace=True)

In [55]:
ventas.drop(columns=['delivery_date', 'Date'], inplace=True)

In [56]:
ventas.head(1)

nro_orden      fecha                           producto  cantidad  precio  \
0  18120194 2023-01-03  Papa Blanca Sucia Tamaño Mixto Kg        10  2750.0   

   descuento                           customer_id  \
0    46750.0  1f1b98af-ee04-4849-a6cf-4fecdf7dba50   

                             sku product_id  product_quantity_x_step_unit  \
0  BAQ-FRU1-CAT6-234:304:750:770        770                           100   

   product_step_unit product_unit sku_parent  month  totalVentasSinDescuento  \
0                 10     Kilogram   No value      1                  27500.0   

   totalVentasConDescuento  unit cost  
0                 -19250.0     2330.0

Observemos que aun despues de el merge, existen nulos en los costos. Esto puede deberse a que estos sku fueron costeados después de la franja de tiempo acotada.

In [57]:
ventas.sku[ventas['unit cost'].isnull()].unique()

array(['BAQ-FRU1-CAT1-59:90:201:202',
       'BAQ-FRU1-CAT2-13447:24531:24532:16342',
       'BAQ-FRU1-CAT1-241:367:958:959',
       'BAQ-FRU1-CAT104105-305509:1018263:1018264:563300',
       'BAQ-FRU1-CAT2-281:370:964:965',
       'BAQ-FRU1-CAT2-36215:63238:63239:42882',
       'BAQ-FRU1-CAT6-298:790:2017:2018',
       'BAQ-FRU1-CAT1-44789:80549:80550:53091',
       'BAQ-FRU1-CAT104111-60301:124347:124348:73260',
       'BAQ-FRU1-CAT2-6:45:106:107',
       'BAQ-FRU1-CAT104105-104967:239539:239540:131193',
       'BAQ-FRU1-CAT1-208:432:1113:1114', 'BAQ-FRU1-CAT6-283:371:966:967',
       'BAQ-FRU1-CAT1-440:579:1507:1508', 'BAQ-FRU1-CAT1-52:78:177:178',
       'BAQ-FRU1-CAT104111-64:1114193:1114194:608238',
       'BAQ-FRU1-CAT2-264:351:907:908', 'BAQ-FRU1-CAT1-463:720:1869:1870',
       'BAQ-FRU1-CAT2-280:368:960:961', 'BAQ-FRU1-CAT6-658:852:2157:2158',
       'BAQ-FRU1-CAT1-47:68:235:236', 'BAQ-FRU1-CAT1-60:91:203:204',
       'BAQ-FRU1-CAT1-236:307:760:761', 'BAQ-FRU1-CAT2-32:65:725:7

In [58]:
null_product = list(ventas.producto[ventas['unit cost'].isnull()].unique())

In [59]:
ventas = ventas[ventas['unit cost'].notnull()]

Ahora, nos queda agregar algunos calculos adicionales.

In [60]:
ventas['totalContribucionSinDescuento'] = (ventas.precio - ventas['unit cost']) * ventas.cantidad
ventas['totalContribucionConDescuento'] = ((ventas.precio - ventas['unit cost']) * ventas.cantidad) - ventas.descuento

Añadimos los valores de la tabla de producto

In [61]:
ventasCopy2 = ventas.copy()

In [62]:
productos.head(2)

product_id                            sku                      name  \
0        660  BAQ-FRU1-CAT1-111:277:659:660  Granadilla Estándar - Kg   
1        676  BAQ-FRU1-CAT1-123:280:665:676        Kiwi Estándar - Kg   

  category region_code product_category_id  mean_shelf_life  \
0   Frutas         BAQ                   1                2   
1   Frutas         BAQ                   1                2   

   promised_lead_time  purchasing_unit buy_unit  weight_parameter_apricot  
0                 NaN              1.0       KG                       1.0  
1                 NaN              1.0       KG                       1.0

In [63]:
ventas = pd.merge(ventas, productos, how='left', on='sku')

In [64]:
ventas.drop(columns=['product_id_y'], inplace=True)

Al parecer la base de datos de productos no tenía información de algunos sku.

In [67]:
ventas.to_csv('../Datos Externos/Interim/base_consolidada.csv')